<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [12]</a>'.</span>

importing models

In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

2024-09-16 02:27:24.212716: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-16 02:27:24.224022: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-16 02:27:24.301543: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-16 02:27:24.381382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-16 02:27:24.492434: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

2024-09-16 02:27:26.558274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Ratings data.
ratings = tfds.load("movielens/latest-small-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/latest-small-movies", split="train")

2024-09-16 02:27:28.330238: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/9742 [00:00<?, ? examples/s]

Shuffling /home/airflow/tensorflow_datasets/movielens/latest-small-movies/incomplete.1PWFMT_0.1.1/movielens-tr…

Dataset movielens downloaded and prepared to /home/airflow/tensorflow_datasets/movielens/latest-small-movies/0.1.1. Subsequent calls will reuse this data.


In [4]:
# Convert to an iterator to view the structure
ratings_iter = ratings.as_numpy_iterator()
movies_iter = movies.as_numpy_iterator()

# Fetch the first example from each dataset
first_rating = next(ratings_iter)
first_movie = next(movies_iter)

# Display the structure of the first example
print("Ratings structure:")
print(first_rating)

print("\nMovies structure:")
print(first_movie)

Ratings structure:
{'movie_genres': array([ 7,  8, 13, 15]), 'movie_id': b'4874', 'movie_title': b'K-PAX (2001)', 'timestamp': 1446749868, 'user_id': b'105', 'user_rating': 5.0}

Movies structure:
{'movie_genres': array([4]), 'movie_id': b'2261', 'movie_title': b'One Crazy Summer (1986)'}


importing mongo data

In [5]:
new_user_movie_titles = ["The Matrix", "Inception", "The Dark Knight", "The Pirates of the Caribbean", "The Lion King"]
new_user_user_ids = ["new_user_123", "new_user_123", "new_user_123","new_user_123","new_user_123"]
# new_user_ratings = [5, 4, 3]

In [6]:
# testing cell
new_user_movie_titles = ['The Godfather', 'The Dark Knight', 'Spirited Away', 'Your Name.', 'The Lord of the Rings: The Return of the King', 'The Lord of the Rings: The Rings of Power', 'Ninjago: Masters of Spinjitzu', 'WWE Main Event', 'Sin senos sí hay paraíso', 'Teen Titans Go!', 'Despicable Me 4', 'The Union', 'Inside Out 2', 'The Convert', 'Dragonkeeper', 'Borderlands', 'Miraculous World: Paris, Tales of Shadybug and Claw Noir', 'Coraline', 'Kung Fu Panda 4', 'The Garfield Movie', 'Inside Out', 'Puss in Boots: The Last Wish', 'The Killer', 'The Good, the Bad and the Ugly', 'Monolith', 'Beetlejuice', 'The Lord of the Rings: The Return of the King', 'Breaking Bad', 'Pacific Rim']
new_user_user_ids = ['66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66d9e483d572df24f50d71cb', '66d9e483d572df24f50d71cb', '66d9e483d572df24f50d71cb', '66d9e483d572df24f50d71cb', '66d9e483d572df24f50d71cb', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66d9e483d572df24f50d71cb', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731', '66de89dd0725860e49c0e59a', '66de89dd0725860e49c0e59a', '66de89dd0725860e49c0e59a', '66de89dd0725860e49c0e59a', '66dab78fd3c14870046e8731', '66dab78fd3c14870046e8731']


In [7]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [8]:
# Convert the separate lists to tensors
new_user_ratings_dataset = tf.data.Dataset.from_tensor_slices({
    "movie_title": tf.constant(new_user_movie_titles),
    "user_id": tf.constant(new_user_user_ids)
})
ratings = ratings.concatenate(new_user_ratings_dataset)

In [9]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [10]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

2024-09-16 02:28:19.559657: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2024-09-16 02:28:22.395417: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


array([b"'71 (2014)", b"'Hellboy': The Seeds of Creation (2004)",
       b"'Round Midnight (1986)", b"'Salem's Lot (2004)",
       b"'Til There Was You (1997)", b"'Tis the Season for Love (2015)",
       b"'burbs, The (1989)", b"'night Mother (1986)",
       b'(500) Days of Summer (2009)', b'*batteries not included (1987)'],
      dtype=object)

In [11]:
unique_user_ids[:10]

array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
       b'106', b'107'], dtype=object)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [12]:
user_ids_vocabulary=tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None)
# user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

ImportError: Keras cannot be imported. Check that it is installed.

In [ ]:
movie_titles_vocabulary=tf.keras.layers.StringLookup(vocabulary=unique_movie_titles, mask_token=None)

In [ ]:
import tensorflow_recommenders as tfrs

In [ ]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [ ]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [ ]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [ ]:


# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
# index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# index.index_from_dataset(
#     movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# # Get some recommendations.
# _, titles = index(np.array(["42"]))
# print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

In [ ]:
class CustomBruteForce(tfrs.layers.factorized_top_k.BruteForce):
    def __init__(self, user_model):
        super().__init__(user_model)
    
    def call(self, inputs, k=100, training=False):  # Add 'training' parameter
        # Call the parent class's call method
        return super().call(inputs, k=k)

In [ ]:
# Create the brute force search index with the default k=100
index = CustomBruteForce(model.user_model)

# Index the movies
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title)))
)

# When you call it without specifying k, it will return 100 results by default
scores, recommended_titles = index(np.array(["66d9e483d572df24f50d71cb"]))

# Print the top 100 recommendations
print(f"Top 100 recommendations: {recommended_titles[0][:100]}")

In [ ]:
_, recommended_titles = index(np.array(["66d9e483d572df24f50d71cb"]),k=100)
print(f"Top 3 recommendations for new_user_123: {recommended_titles[0, :100]}")

In [ ]:
# with tempfile.TemporaryDirectory() as tmp:
#   path = os.path.join(tmp, "model")
# path = 'd:/MovieMania/'
# path  = os.path.join(path, "model")
#   # Save the index.
# tf.saved_model.save(index, path)

In [ ]:
#   # Load it back; can also be done in TensorFlow Serving.
# loaded = tf.saved_model.load(path)

#   # Pass a user id in, get top predicted movie titles back.
# scores, titles = loaded(["66d9e483d572df24f50d71cb"],100)

# print(f"Recommendations: {titles[0][:100]}")